출처: https://ratsgo.github.io/from%20frequency%20to%20semantics/2017/07/09/lda/

In [1]:
from collections import Counter
import random

In [2]:
documents = [["Hadoop", "Big Data", "HBase", "Java", "Spark", "Storm", "Cassandra"],
    ["NoSQL", "MongoDB", "Cassandra", "HBase", "Postgres"],
    ["Python", "scikit-learn", "scipy", "numpy", "statsmodels", "pandas"],
    ["R", "Python", "statistics", "regression", "probability"],
    ["machine learning", "regression", "decision trees", "libsvm"],
    ["Python", "R", "Java", "C++", "Haskell", "programming languages"],
    ["statistics", "probability", "mathematics", "theory"],
    ["machine learning", "scikit-learn", "Mahout", "neural networks"],
    ["neural networks", "deep learning", "Big Data", "artificial intelligence"],
    ["Hadoop", "Java", "MapReduce", "Big Data"],
    ["statistics", "R", "statsmodels"],
    ["C++", "deep learning", "artificial intelligence", "probability"],
    ["pandas", "R", "Python"],
    ["databases", "HBase", "Postgres", "MySQL", "MongoDB"],
    ["libsvm", "regression", "support vector machines"]]

In [3]:
documents

[['Hadoop', 'Big Data', 'HBase', 'Java', 'Spark', 'Storm', 'Cassandra'],
 ['NoSQL', 'MongoDB', 'Cassandra', 'HBase', 'Postgres'],
 ['Python', 'scikit-learn', 'scipy', 'numpy', 'statsmodels', 'pandas'],
 ['R', 'Python', 'statistics', 'regression', 'probability'],
 ['machine learning', 'regression', 'decision trees', 'libsvm'],
 ['Python', 'R', 'Java', 'C++', 'Haskell', 'programming languages'],
 ['statistics', 'probability', 'mathematics', 'theory'],
 ['machine learning', 'scikit-learn', 'Mahout', 'neural networks'],
 ['neural networks', 'deep learning', 'Big Data', 'artificial intelligence'],
 ['Hadoop', 'Java', 'MapReduce', 'Big Data'],
 ['statistics', 'R', 'statsmodels'],
 ['C++', 'deep learning', 'artificial intelligence', 'probability'],
 ['pandas', 'R', 'Python'],
 ['databases', 'HBase', 'Postgres', 'MySQL', 'MongoDB'],
 ['libsvm', 'regression', 'support vector machines']]

In [4]:
random.seed(0)
# topic 수 지정
K = 4

In [5]:
# 각 단어를 임의의 토픽에 랜덤 배정
document_topics = [[random.randrange(K) for word in document]
                   for document in documents]

# 각 토픽이 각 문서에 할당되는 횟수
# Counter로 구성된 리스트
# 각 Counter는 각 문서를 의미
document_topic_counts = [Counter() for _ in documents]

# 각 단어가 각 토픽에 할당되는 횟수
# Counter로 구성된 리스트
# 각 Counter는 각 토픽을 의미
topic_word_counts = [Counter() for _ in range(K)]

# 각 토픽에 할당되는 총 단어의 수
# 숫자로 구성된 리스트
# 각각의 숫자는 각 토픽을 의미
topic_counts = [0 for _ in range(K)]

# 각 문서에 포함되는 총 단어의 수
# 숫자로 구성된 리스트
# 각각의 숫자는 문서를 의미
document_lengths = [len(document) for document in documents]

# 서로 다른 단어의 수
distinct_words = set(word for document in documents for word in document)
V = len(distinct_words)

# 총 문서의 수
D = len(documents)

In [6]:
# 각 문서별 할당된 topic을 랜덤 초기화 후
# AB를 구하는 데 필요한 숫자를 counting
for d in range(D):
    for word, topic in zip(documents[d], document_topics[d]):
        document_topic_counts[d][topic] += 1
        topic_word_counts[topic][word] += 1
        topic_counts[topic] += 1

In [7]:
def choose_new_topic(d, word):
    return sample_from([topic_weight(d, word, k) for k in range(K)])

In [8]:
def topic_weight(d, word, k):
    # 문서와 문서의 단어가 주어지면
    # k번째 토픽의 weight를 반환
    return p_word_given_topic(word, k) * p_topic_given_document(k, d)

In [9]:
def p_word_given_topic(word, topic, beta=0.1):
    # topic에 속한 단어 가운데 word의 비율
    # (beta를 더해 smoothing)
    return ((topic_word_counts[topic][word] + beta) /
           (topic_counts[topic] + V * beta))

In [10]:
def p_topic_given_document(topic, d, alpha=0.1):
    # 문서 d의 모든 단어 가운데 topic에 속하는
    # 단어의 비율 (alpha를 더해 smoothing)
    return ((document_topic_counts[d][topic] + alpha) /
            (document_lengths[d] + K * alpha))

In [11]:
def sample_from(weights):
    # i를 확률로 반환
    total = sum(weights)
    # 0과 total 사이를 균일하게 선택
    rnd = total * random.random()
    # 아래 식을 만족하는 가장 작은 i를 반환
    # weights[0] + ... + weights[i] >= rnd
    for i, w in enumerate(weights):
        rnd -= w
        if rnd <= 0:
            return i

In [12]:
for _ in range(10000):
    for d in range(D):
        for i, (word, topic) in enumerate(zip(documents[d],
                                              document_topics[d])):
            # 깁스 샘플링 수행을 위해
            # 샘플링 대상 word와 topic을 제외하고 세어봄
            document_topic_counts[d][topic] -= 1
            topic_word_counts[topic][word] -= 1
            topic_counts[topic] -= 1
            document_lengths[d] -= 1
            
            # 깁스 샘플링 대상 word와 topic을 제외한
            # 말뭉치 모든 word의 topic 정보를 토대로
            # 샘플링 대상 word의 새로운 topic을 선택
            new_topic = choose_new_topic(d, word)
            document_topics[d][i] = new_topic
            
            # 샘플링 대상 word의 새로운 topic을 반영해
            # 말뭉치 정보 업데이트
            document_topic_counts[d][new_topic] += 1
            topic_word_counts[new_topic][word] += 1
            topic_counts[new_topic] += 1
            document_lengths[d] += 1

In [13]:
document_topic_counts

[Counter({3: 0, 0: 0, 2: 2, 1: 5}),
 Counter({3: 0, 2: 5, 1: 0, 0: 0}),
 Counter({1: 0, 0: 1, 2: 1, 3: 4}),
 Counter({0: 5, 2: 0, 3: 0, 1: 0}),
 Counter({3: 3, 2: 0, 1: 0, 0: 1}),
 Counter({3: 0, 2: 0, 0: 3, 1: 3}),
 Counter({0: 4, 3: 0, 2: 0, 1: 0}),
 Counter({2: 0, 0: 0, 1: 0, 3: 4}),
 Counter({1: 3, 3: 1, 0: 0, 2: 0}),
 Counter({0: 0, 2: 0, 3: 0, 1: 4}),
 Counter({2: 2, 0: 1, 3: 0, 1: 0}),
 Counter({2: 0, 1: 2, 3: 0, 0: 2}),
 Counter({0: 2, 3: 0, 2: 1, 1: 0}),
 Counter({1: 0, 2: 5, 3: 0, 0: 0}),
 Counter({0: 0, 2: 0, 3: 3, 1: 0})]

In [14]:
topic_word_counts[0]

Counter({'HBase': 0,
         'scikit-learn': 0,
         'pandas': 0,
         'R': 3,
         'regression': 1,
         'Java': 0,
         'C++': 1,
         'Haskell': 0,
         'statistics': 3,
         'artificial intelligence': 0,
         'Hadoop': 0,
         'Big Data': 0,
         'statsmodels': 0,
         'libsvm': 0,
         'Spark': 0,
         'Storm': 0,
         'programming languages': 0,
         'machine learning': 0,
         'MapReduce': 0,
         'scipy': 0,
         'numpy': 0,
         'support vector machines': 0,
         'Cassandra': 0,
         'deep learning': 1,
         'decision trees': 1,
         'neural networks': 0,
         'databases': 0,
         'probability': 3,
         'theory': 1,
         'NoSQL': 0,
         'Mahout': 0,
         'mathematics': 1,
         'Postgres': 0,
         'Python': 4,
         'MySQL': 0,
         'MongoDB': 0})